In [1]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
import pickle

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
data = pd.read_csv("feature.csv")

In [5]:
data.head()

,Unnamed: 0,File,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,has_ip,...,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,0,spam_dataset,31,0.064516,0,0,-4.708307,True,False,0,...,0,0,0,0,0.000000,0,2,0.000000,False,81
1,1,spam_dataset,64790,0.306050,0,0,-4.576882,True,False,0,...,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,2,spam_dataset,757,0.195509,0,0,-4.205536,True,False,0,...,0,1,0,0,0.000000,0,148,0.000000,False,46
3,3,spam_dataset,266310,0.230915,0,0,-4.550931,True,False,0,...,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,4,spam_dataset,40,0.200000,0,0,-4.689336,True,False,0,...,2,1,0,0,0.000000,0,8,0.000000,False,83


In [6]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [7]:
data = data.drop(columns = "File")

In [8]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [26]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [28]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
224/224 [==============================] - 1s 3ms/step - loss: 1.3426 - accuracy: 0.4081 - val_loss: 0.9693 - val_accuracy: 0.6035
Epoch 2/50
224/224 [==============================] - 0s 2ms/step - loss: 0.9376 - accuracy: 0.6241 - val_loss: 0.8951 - val_accuracy: 0.6361
Epoch 3/50
224/224 [==============================] - 0s 2ms/step - loss: 0.8558 - accuracy: 0.6594 - val_loss: 0.7871 - val_accuracy: 0.6844
Epoch 4/50
224/224 [==============================] - 0s 2ms/step - loss: 0.7463 - accuracy: 0.7051 - val_loss: 0.7288 - val_accuracy: 0.7156
Epoch 5/50
224/224 [==============================] - 0s 2ms/step - loss: 0.7151 - accuracy: 0.7159 - val_loss: 0.6656 - val_accuracy: 0.7431
Epoch 6/50
224/224 [==============================] - 0s 2ms/step - loss: 0.6645 - accuracy: 0.7385 - val_loss: 0.6523 - val_accuracy: 0.7408
Epoch 7/50
224/224 [==============================] - 0s 2ms/step - loss: 0.6392 - accuracy: 0.7510 - val_loss: 0.6171 - val_accuracy: 0.7617
Epoch 

In [29]:
y_pred = model.predict(X_test)

In [30]:
predicted = np.argmax(y_pred, axis=1)

In [31]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8459620039609771


In [32]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.94      0.95      0.95      2735
  Defacement       0.72      0.79      0.76      2499
     Malware       0.82      0.92      0.87      2834
    Phishing       0.83      0.69      0.75      2477
        Spam       0.91      0.86      0.88      3088

    accuracy                           0.85     13633
   macro avg       0.85      0.84      0.84     13633
weighted avg       0.85      0.85      0.85     13633



In [34]:
os.chdir("../")
os.chdir("models")
model.save("Model_v1.h5")
np.save('lblenc.npy', encoder.classes_)
scalerfile = 'scaler.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))